In [ ]:
import requests
import json

def upload_file(file_path, url='http://127.0.0.1:8000/v1/files'):
    """
    Upload a file to the specified URL endpoint.

    Args:
        file_path (str): Path to the file to be uploaded
        url (str): URL endpoint for file upload (default: http://127.0.0.1:8000/v1/files)

    Returns:
        requests.Response: Response from the server
    """
    with open(file_path, 'rb') as file:
        files = {'file': file}
        response = requests.post(url, files=files)
    return response

# Example usage:
# response = upload_file('./training_data_sft.jsonl')
response = upload_file('tests/data/training_data_sft.jsonl')
print(response.status_code)  # Print the HTTP status code
response_body = json.loads(response.text)
uploaded_file = response_body["id"]
print(response_body)


200
{'id': 'ef1b1eb5-be05-4783-a412-dc2e36afaba2', 'object': 'file', 'bytes': 1227, 'created_at': 1741705972, 'filename': 'training_data_sft.jsonl', 'purpose': 'training'}


In [7]:
#"HuggingFaceTB/SmolLM2-135M-Instruct"
#"Qwen/Qwen2-0.5B-Instruct"
def start_fine_tune(training_file, model_name="HuggingFaceTB/SmolLM2-135M-Instruct", url='http://127.0.0.1:8000/v1/fine_tuning/jobs'):
    """
    Start a fine-tuning job with the specified training file and model.

    Args:
        training_file (str): ID of the uploaded training file
        model_name (str): Name of the base model to fine-tune (default: Qwen/Qwen2-0.5B-Instruct)
        url (str): URL endpoint for fine-tuning (default: http://127.0.0.1:8000/v1/fine_tuning/jobs)

    Returns:
        requests.Response: Response from the server
    """
    headers = {'Content-Type': 'application/json'}
    data = {
        'model': model_name,
        'training_file': training_file
    }
    response = requests.post(url, headers=headers, json=data)
    return response

# Example usage (continuing from previous upload):
fine_tune_response = start_fine_tune(uploaded_file)
print(fine_tune_response.status_code)
print(fine_tune_response.text)

500
Internal Server Error


In [6]:
def check_job_status(job_id, url_base='http://127.0.0.1:8000/v1/fine_tuning/jobs'):
    """
    Check the status of a fine-tuning job.

    Args:
        job_id (str): ID of the fine-tuning job
        url_base (str): Base URL for job status endpoint (default: http://127.0.0.1:8000/v1/fine_tuning/jobs)

    Returns:
        requests.Response: Response from the server
    """
    url = f"{url_base}/{job_id}"
    response = requests.get(url)
    return response

# Example usage (continuing from previous fine-tune):
job_response = check_job_status(fine_tune_response.json()["id"])
print(job_response.status_code)
print(job_response.text)

200
{"id":"bdcc5b2f-b46c-41be-a00c-3671793700f6","status":"succeeded","details":"","fine_tuned_model":"/home/noah/Code/OSS/arbor/storage/models/ft:smollm2-135m-instruct:inhvr6:20250311_111457"}


In [58]:
def load_model(model_path, device=None):
    """
    Load a fine-tuned model from HuggingFace.

    Args:
        job_response: Response from check_job_status() containing model path
        device: Device to load model on (default: None, will use CUDA if available)

    Returns:
        tuple: (model, tokenizer) loaded from the fine-tuned path
    """
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer

    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # if not model_path:
        # raise ValueError("No fine-tuned model path found in job response")

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map=device,
        torch_dtype=torch.float16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    return model, tokenizer

# Example usage (continuing from previous job check):
model, tokenizer = load_model(job_response.json()['fine_tuned_model'])


In [10]:
from openai import OpenAI

client = OpenAI(
    base_url="http://127.0.0.1:8000/v1",  # Using Arbor server
    api_key="not-needed"  # If you're using a local server, you dont need an API key
)

In [11]:
data_file = 'tests/data/training_data_sft.jsonl'#'./training_data_sft.jsonl'
file = client.files.create(
  file=open(data_file, "rb"),
  purpose="fine-tune"
)
file

FileObject(id='file-06281957-4f3d-4579-9cc6-c014c88eb6a2', bytes=1227, created_at=1742251912, filename='training_data_sft.jsonl', object='file', purpose='training', status=None, expires_at=None, status_details=None)

In [12]:
job = client.fine_tuning.jobs.create(
  training_file=file.id,
  model="HuggingFaceTB/SmolLM2-135M-Instruct"
)
job

FineTuningJob(id='ftjob-a890525c-281e-4c64-b384-666ea0345722', created_at=None, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=None, model=None, object='fine_tuning.job', organization_id=None, result_files=None, seed=None, status='queued', trained_tokens=None, training_file=None, validation_file=None, estimated_finish=None, integrations=None, metadata=None, method=None)

In [13]:
job_update = client.fine_tuning.jobs.retrieve(job.id)
print(f"Job status: {job_update.status}")
print(f"Fine-tuned model: {job_update.fine_tuned_model}")


Job status: succeeded
Fine-tuned model: /home/noah/Code/OSS/arbor/storage/models/ft:smollm2-135m-instruct:Wz3blF:20250317_185154


In [15]:
completion = client.chat.completions.create(
  model="/home/noah/Code/OSS/arbor/storage/models/ft:smollm2-135m-instruct:Wz3blF:20250317_185154",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

APIConnectionError: Connection error.